<a href="https://colab.research.google.com/github/heyggun/bigdata_analysis_test/blob/main/DataManim_type2_Classification_3_hyperparametertuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형 2유형

https://www.datamanim.com/dataset/03_dataq/typetwo.html#id5

## 분류 classification 

### 정시 배송 여부 판단

- 데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)
- x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv
- y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv
- x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv
- x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv
- 데이터 출처 :https://www.kaggle.com/datasets/prachi13/customer-analytics (참고, 데이터 수정)

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

In [2]:
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv')

x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv')


In [3]:
display(x_train.head(3))
print(x_train.shape)
display(y_train.head(3))
print(y_train.shape)

display(x_test.head(3))
print(x_test.shape)
display(y_test.head(3))
print(y_test.shape)

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674


(6598, 11)


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1


(6598, 2)


,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6811,D,Ship,5,2,259,5,low,F,7,1032
1,4320,F,Ship,3,5,133,3,medium,F,4,5902
2,5732,F,Road,3,4,191,5,medium,F,4,4243


(4401, 11)


,ID,Reached.on.Time_Y.N
0,6811,0
1,4320,0
2,5732,1


(4401, 2)


In [4]:
x_tr = x_train.drop(columns=['ID'])
y_tr = y_train.drop(columns=['ID'])

x_te = x_test.drop(columns=['ID'])
y_te = y_test.drop(columns=['ID'])

In [5]:
obj = x_tr.select_dtypes(include='object').columns
num = x_tr.select_dtypes(include='number').columns

print(obj)
print(num)

print(len(obj) + len(num))
print(len(x_tr.columns))

display(x_tr[obj].head(3))
display(x_tr[num].head(3))


Index(['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Product_importance', 'Gender'],
      dtype='object')
Index(['Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Discount_offered', 'Weight_in_gms'],
      dtype='object')
10
10


,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Product_importance,Gender
0,A,Flight,4,high,F
1,F,Ship,3,low,M
2,F,Road,4,high,M


,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
0,3,266,5,5,1590
1,1,174,2,44,1556
2,1,154,10,10,5674


In [6]:
temp = pd.concat([x_tr.assign(ind='train'), x_te.assign(ind='test')])
temp.head(3)

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,ind
0,A,Flight,4,3,266,5,high,F,5,1590,train
1,F,Ship,3,1,174,2,low,M,44,1556,train
2,F,Road,4,1,154,10,high,M,10,5674,train


- numeric한 column 인데 object으로 된 column 변환

In [7]:
temp['Customer_care_calls'].value_counts()

4     3557
3     3217
5     2328
6     1013
2      638
$7     246
Name: Customer_care_calls, dtype: int64

In [8]:
temp['Customer_care_calls'] = temp['Customer_care_calls'].str.replace('$','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [9]:
temp['Customer_care_calls'] = temp['Customer_care_calls'].astype(int)

- 범주형 변수 encoding

In [10]:
print(obj)

for col in obj:
  print(x_tr[col].unique())
  print(x_tr[col].nunique())

Index(['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Product_importance', 'Gender'],
      dtype='object')
['A' 'F' 'C' 'B' 'D']
5
['Flight' 'Ship' 'Road']
3
['4' '3' '5' '6' '2' '$7']
6
['high' 'low' 'medium']
3
['F' 'M']
2


- 범주형 변수 OneHotEncoding 하자

In [11]:
temp = pd.get_dummies(temp)
display(temp.head(3))

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,...,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_F,Gender_M,ind_test,ind_train
0,4,3,266,5,5,1590,1,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1,3,1,174,2,44,1556,0,0,0,0,...,0,0,1,0,1,0,0,1,0,1
2,4,1,154,10,10,5674,0,0,0,0,...,0,1,0,1,0,0,0,1,0,1


In [12]:
x_tr = temp.loc[temp['ind_train']==1].drop(columns=['ind_train','ind_test'])
x_te = temp.loc[temp['ind_test']==1].drop(columns=['ind_train','ind_test'])

print(x_tr.shape)
print(x_te.shape)

(6598, 19)
(4401, 19)


- 수치형 scaling 안하고 모델링

In [13]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(x_tr,y_tr,
                                                            test_size=0.3,
                                                            random_state=2023)

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

(4618, 19)
(1980, 19)
(4618, 1)
(1980, 1)


In [14]:
model = RandomForestClassifier()
model.fit(X_train_m, y_train_m)
pred = model.predict_proba(X_test_m)[:,1]
pred_class = model.predict(X_test_m)

print(roc_auc_score(y_test_m, pred))
print(f1_score(y_test_m, pred_class))
print(accuracy_score(y_test_m, pred_class))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.7433454322256725
0.689968225147526
0.6550505050505051


In [15]:
pred = model.predict_proba(x_te)[:,1]
pred_class = model.predict(x_te)

print(roc_auc_score(y_te, pred))
print(f1_score(y_te, pred_class))
print(accuracy_score(y_te, pred_class))

0.734792272293318
0.6884102564102564
0.6548511701885935


- 수치형 scaling 해보기

In [16]:
def do_scaling(df):
  cols = ['Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Discount_offered', 'Weight_in_gms','Customer_care_calls']

  for col in cols:
    Scaler = StandardScaler()
    df[col] = Scaler.fit_transform(df[col].values.reshape(-1,1))

  return df

x_tr_s = do_scaling(x_tr)
x_te_s = do_scaling(x_te)

display(x_tr_s.head(3))
display(x_te_s.head(3))

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_F,Gender_M
0,-0.045238,0.006129,1.152368,0.941089,-0.516099,-1.231489,1,0,0,0,0,1,0,0,1,0,0,1,0
1,-0.915436,-1.412796,-0.754193,-1.043983,1.893385,-1.252277,0,0,0,0,1,0,0,1,0,1,0,0,1
2,-0.045238,-1.412796,-1.168663,4.249543,-0.207190,1.265494,0,0,0,0,1,0,1,0,1,0,0,0,1


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_F,Gender_M
0,0.833667,-0.696928,1.027827,0.940162,-0.394425,-1.619908,0,0,0,1,0,0,0,1,0,1,0,1,0
1,-0.936643,1.416425,-1.609909,-0.358740,-0.579236,1.360732,0,0,0,0,1,0,0,1,0,0,1,1,0
2,-0.936643,0.711974,-0.395713,0.940162,-0.579236,0.345356,0,0,0,0,1,0,1,0,0,0,1,1,0


In [17]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(x_tr_s,y_tr,
                                                            test_size=0.3,
                                                            random_state=2023)

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

(4618, 19)
(1980, 19)
(4618, 1)
(1980, 1)


In [18]:
model = RandomForestClassifier()
model.fit(X_train_m, y_train_m)
pred = model.predict_proba(X_test_m)[:,1]
pred_class = model.predict(X_test_m)

print(roc_auc_score(y_test_m, pred))
print(f1_score(y_test_m, pred_class))
print(accuracy_score(y_test_m, pred_class))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.7413776808356094
0.7040862146385272
0.6671717171717172


In [19]:
pred = model.predict_proba(x_te_s)[:,1]
pred_class = model.predict(x_te_s)

print(roc_auc_score(y_te, pred))
print(f1_score(y_te, pred_class))
print(accuracy_score(y_te, pred_class))

0.738839878570739
0.6963923794081881
0.6596228129970462


In [20]:
def do_scaling(df):
  cols = ['Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Discount_offered', 'Weight_in_gms','Customer_care_calls']

  for col in cols:
    Scaler = MinMaxScaler()
    df[col] = Scaler.fit_transform(df[col].values.reshape(-1,1))

  return df

x_tr_m = do_scaling(x_tr)
x_te_m = do_scaling(x_te)

display(x_tr_m.head(3))
display(x_te_m.head(3))

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_F,Gender_M
0,0.4,0.5,0.794393,0.375,0.062500,0.088134,1,0,0,0,0,1,0,0,1,0,0,1,0
1,0.2,0.0,0.364486,0.000,0.671875,0.083047,0,0,0,0,1,0,0,1,0,1,0,0,1
2,0.4,0.0,0.271028,1.000,0.140625,0.699237,0,0,0,0,1,0,1,0,1,0,0,0,1


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_F,Gender_M
0,0.6,0.25,0.761682,0.375,0.093750,0.004529,0,0,0,1,0,0,0,1,0,1,0,1,0
1,0.2,1.00,0.172897,0.125,0.046875,0.715997,0,0,0,0,1,0,0,1,0,0,1,1,0
2,0.2,0.75,0.443925,0.375,0.046875,0.473630,0,0,0,0,1,0,1,0,0,0,1,1,0


In [21]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(x_tr_m,y_tr,
                                                            test_size=0.3,
                                                            random_state=2023)

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

(4618, 19)
(1980, 19)
(4618, 1)
(1980, 1)


In [22]:
model = RandomForestClassifier()
model.fit(X_train_m, y_train_m)
pred = model.predict_proba(X_test_m)[:,1]
pred_class = model.predict(X_test_m)

print(roc_auc_score(y_test_m, pred))
print(f1_score(y_test_m, pred_class))
print(accuracy_score(y_test_m, pred_class))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.7437542742576065
0.6989588048890901
0.6641414141414141


In [23]:
pred = model.predict_proba(x_te_m)[:,1]
pred_class = model.predict(x_te_m)

print(roc_auc_score(y_te, pred))
print(f1_score(y_te, pred_class))
print(accuracy_score(y_te, pred_class))

0.7365119122963217
0.6866518128418069
0.6484889797773233


**교차검증 + 하이퍼파라미터 튜닝**

In [37]:
skf = StratifiedKFold(n_splits=10, random_state=2023, shuffle=True)

In [38]:
len(x_tr.columns)

19

In [39]:
model = RandomForestClassifier()
model_params = {'max_depth':[5,6,7], 'max_features':[5,10,15,19], 'n_estimators':[10,15,20]}

In [40]:
rand_cv = RandomizedSearchCV(model, model_params, cv=skf, verbose=-1, n_jobs=-1)
rand_cv.fit(x_tr, y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=2023, shuffle=True),
                   estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [5, 6, 7],
                                        'max_features': [5, 10, 15, 19],
                                        'n_estimators': [10, 15, 20]},
                   verbose=-1)

In [41]:
rand_cv.best_estimator_

RandomForestClassifier(max_depth=6, max_features=10, n_estimators=10)

In [42]:
model_pred = np.zeros((x_te.shape[0],2))
model_class =[]

for train_idx, test_idx in skf.split(x_tr, y_tr):
  X_train_m, X_test_m = x_tr.iloc[train_idx], x_tr.iloc[test_idx]
  y_train_m, y_test_m = y_tr.iloc[train_idx], y_tr.iloc[test_idx]

  model = rand_cv.best_estimator_
  model.fit(X_train_m, y_train_m)
  fold_pred = model.predict_proba(x_te)/10
  fold_class =model.predict(x_te)

  print(fold_class)

  model_pred += fold_pred
  model_class.append(fold_class)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


[0 0 0 ... 0 0 1]
[0 0 0 ... 0 0 1]
[0 0 0 ... 0 0 1]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


[0 0 0 ... 0 0 1]
[1 0 0 ... 0 0 1]
[0 0 0 ... 0 0 1]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


[0 0 0 ... 0 0 1]
[0 0 0 ... 0 0 1]
[0 0 0 ... 0 0 1]
[0 0 0 ... 0 0 1]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


In [43]:
pred = np.argmax(model_pred, axis= 1)
pred

array([0, 0, 0, ..., 0, 0, 1])

In [44]:
print(roc_auc_score(y_te, model_pred[:,1]))
print(f1_score(y_te, np.argmax(model_pred, axis=1)))
print(accuracy_score(y_te, np.argmax(model_pred, axis=1)))

0.7367285970200487
0.6545718432510885
0.6755282890252216
